# Exercise 3

task description

In [1]:
import numpy as np # linear algebra
import pandas as pd
import tensorflow as tf

In [ ]:
data = pd.read_csv("data/ner/gmb.csv",encoding = 'latin1')
data = data.fillna(method = 'ffill')
data.head()

In [ ]:
class Get_sentence(object):
    def __init__(self,data):
        self.n_sent=1
        self.data = data
        agg_func = lambda s:[(w,p,t) for w,p,t in zip(s["Word"].values.tolist(),
                                                     s["POS"].values.tolist(),
                                                     s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [ ]:
getter = Get_sentence(data)
sentence = getter.sentences
sentence[0]

In [ ]:
words = list(set(data["Word"].values))
words.append("ENDPAD")

words_tag = list(set(data["Tag"].values))

word_idx = {w : i+1 for i ,w in enumerate(words)}
tag_idx =  {t : i for i ,t in enumerate(words_tag)}

num_words = len(words)
num_words_tag = len(words_tag)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

max_len = 50
X = [[word_idx[w[0]] for w in s] for s in sentence]
X = pad_sequences(maxlen = max_len,sequences = X,padding = 'post',value = num_words-1)
y = [[tag_idx[w[2]] for w in s] for s in sentence]
y = pad_sequences(maxlen = max_len,sequences = y,padding = 'post',value = tag_idx['O'])
y = [to_categorical(i,num_classes = num_words_tag) for i in  y]

In [ ]:
X[1:10]

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.1,random_state=1)

In [ ]:
from tensorflow.keras import Model,Input
from tensorflow.keras.layers import LSTM,Embedding,Dense
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D,Bidirectional
from tensorflow.python.keras import models

In [ ]:
model = models.Sequential()
model.add(Input(shape = (max_len,)))
model.add(Embedding(input_dim = num_words,output_dim = max_len,input_length = max_len))
model.add(SpatialDropout1D(0.1))
model.add(Bidirectional(LSTM(units=100,return_sequences = True, recurrent_dropout = 0.1)))
model.add(TimeDistributed(Dense(num_words_tag,activation = 'softmax')))
model.summary()

In [ ]:
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping 
# from livelossplot import PlotLossesKeras
early_stopping = EarlyStopping(monitor = 'val_accuracy',patience =2,verbose = 0,mode = 'max',restore_best_weights = False)
callbacks = [early_stopping]

history = model.fit(
    x_train,np.array(y_train),
    validation_split =0.2,
    batch_size = 64,
    epochs = 3,
    verbose =1
)

In [ ]:
model.evaluate(x_test,np.array(y_test))

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

Y_test = np.argmax(y_test, axis=1)
y_pred = model.predict_classes(x_test)

In [ ]:
Y_test

## Challenge


In [ ]:
! curl https://nilsreiter.de/challenge.wb.csv > data/ner/challenge.wb.csv

In [ ]:
challenge = pd.read_csv("data/ner/challenge.wb.csv", header = 0, names=["Sentence #","Word","POS","Tag"])


In [ ]:
challenge

In [ ]:
getter = Get_sentence(challenge)
sentence = getter.sentences
sentence[0]

In [ ]:
max_len = 50
x_challenge = [[word_idx.get(w[0],0) for w in s] for s in sentence]
x_challenge = pad_sequences(maxlen = max_len,sequences = x_challenge,padding = 'post',value = num_words-1)
y_challenge = [[tag_idx[w[2]] for w in s] for s in sentence]
y_challenge = pad_sequences(maxlen = max_len,sequences = y_challenge,padding =
                        'post',value = tag_idx['O'])
y_challenge = [to_categorical(i,num_classes = num_words_tag) for i in  y_challenge]

In [ ]:
y_challenge[1:10]

In [ ]:
model.evaluate(x_challenge, np.array(y_challenge))